### Clustering using PseAAC features

##### <u> Part 1: Extracting protein sequence data </u>

In [2]:
import re

class CAZy_data:
    def __init__(self,filename1,filename2):
        self.data,self.acc,self.seq=[],[],[]
        with open(filename1,'r',encoding='utf-8') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split('$'))
        with open(filename2,'r',encoding='utf-8') as inpt1:
            for each1 in inpt1:
                if each1.startswith('>'):
                    self.acc.append(each1.rstrip())
                else:
                    self.seq.append(each1.rstrip())
            
                  
    def data_fetch(self,typ,position):
        typ_data=[]
        if typ=='all':
            typ_data=self.data
        else:
            for each in self.data:
                mult=each[position].split(' ')
                if len(mult)==1:#### In case typ = EC, Multi EC number and protein with no EC number are ignore.
                    if mult[0]==typ:
                        typ_data.append(each)
        return typ_data
    
    def EC_GH(self,ec_no,gh_fam):
        self.fasta=[]
        cazy_ec=self.data_fetch(ec_no,1)
        cazy_gh=self.data_fetch(gh_fam,-1)
        self.common_data=[[i[0],i[1],i[3],i[4],i[-2],i[-1]] for i in cazy_ec if i in cazy_gh]
        rm_prt, rm_prt_fasta=[],[]
        for each in range(len(self.common_data)):
            t=self.common_data[each]
            if self.prtn_filter(t[0]):
                all_acc=t[3].split(' ')
                if all_acc[0]!='':
                    for e_acc in all_acc:
                        e_seq=self.seq_fetch(e_acc)
                        try:
                            create_error=0/len(e_seq) # to remove accession number which doesnt have hits
                            self.fasta.append(f'>{e_acc}${t[0]}${t[1]}${t[2]}${t[-2]}${t[-1]}')
                            self.fasta.append(e_seq[0])
                        except ZeroDivisionError:
                            rm_prt_fasta.append(e_acc)
            else:
                rm_prt.append(t)
#         print('Total number of sequences:',len(self.fasta)/2)
#         print('Number of removed partial or fragment proteins (CAZy):',len(rm_prt))
#         print('Number of removed partial or fragment proteins (Fasta):',len(rm_prt_fasta))
        return self.fasta,rm_prt,rm_prt_fasta
    def prtn_filter(self,prt_name):
        hit=1
        if re.search('partial|fragment',prt_name.lower()):
            hit-=1
        return hit
            
    def seq_fetch(self,accession):
        hits=[]
        temp=0
        for each in range(len(self.acc)):
            if re.search(f'{accession}\D',self.acc[each]):
                temp+=1
                
                if self.prtn_filter(self.acc[each]): # remove partial| fragment accession numbers from GenBank description
                    hits.append(self.seq[each])
        if temp>1:
            print(f'Multiple hits for {accession}')
        elif temp==0:
            print(f'No hits for {accession}')
        return hits
     
In_data=CAZy_data('D:/After_4_4_22/Project_1_Part_1/CAZy_23_6_22/cazy_char_10_6_22.txt','D:/After_4_4_22/Project_1_Part_1/CAZy_23_6_22/char_gh_23_6_22.txt')

##### <u> Part 2: Extracting feature from protein sequences </u>

In [25]:
import numpy as np
from scipy import stats
import pandas as pd

# 1: Hydrophobicity, 2: Hydrophilicity, 3: mass, 4: pk1, 5:pk2, 6:pi, 20: 14 scale, 60: Tanford
class Standard_values:
    def __init__(self,filename):
        self.data=[]
        with open(filename,'r') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split(','))
        del self.data[0]
        
    def get_prop(self,prop):
        got_prop,amino_acid={},{}
        for each in prop:
            got_prop[each]=self.properties(each)
        d=1
        for each in got_prop[1].keys():
            amino_acid[d]=each
            d+=1
        return amino_acid,got_prop
    
    def properties(self, val):
        temp,norm={},{}
        for each in self.data:
            temp[each[0]]=float(each[val])
        relative=stats.zscore(np.array(list(temp.values())))
        for a,b in zip(temp.keys(),relative):
            norm[a]=b
        return norm

class Sequence:
    def __init__(self,filename):
        self.data=[]
        if type(filename)==str:
            with open(filename,'r') as inpt:
                for each in inpt:
                    self.data.append(each.rstrip())
        else:
            self.data=filename
                       
    def output(self):
        a,s,l=[],[],[]
        unusual=0
        no_dup,dup=self.remove_dup(self.ml_sl())
        sig_fam,mlti=self.multi_fam(no_dup)
        for ele in sig_fam:
            if ele.startswith('>'):
                a.append(ele)
            else:
                if re.search('[UZOBJX]',ele.upper()):
#                     print(a[-1])
                    del a[-1]
                    unusual+=1
                    continue
                l.append(len(ele))
                s.append(ele.upper())
#         print('The length of the smallest sequence:',min(l))
#         print('Sequence with "X" present:',unusual)
        return a,s,l,unusual,dup,mlti
    
    def multi_fam(self,duta): # remove the same accession number present in multiple families
        temp=defaultdict(list)
        for ii in range(0,len(duta),2):
            name=duta[ii].split('$')[0][1:]
            temp[name].append([duta[ii],duta[ii+1]])
            
        new_duta,mut=[],[]
        for jj,kk in dict(temp).items():
            if len(kk)!=1:
                mut.append(jj)
            else:
                new_duta.append(kk[0][0])
                new_duta.append(kk[0][1])
        return new_duta,mut

    def remove_dup(self,datu): # remove the duplicates 
        temp_acc,temp_seq,temp_acc_seq=[],[],[]
        dup=[]  
        for iii in range(0,len(datu),2):
            if datu[iii] not in temp_acc:
                temp_acc.append(datu[iii])
                temp_acc_seq.append(datu[iii])
                temp_acc_seq.append(datu[iii+1])
            else:
                dup.append(datu[iii])
        return temp_acc_seq,len(dup)
               
    def ml_sl(self):
        acc_seq=[]
        for k in range(len(self.data)):
            if self.data[k].startswith('>'):
                acc_seq.append(self.data[k])
                join_=0
                for l in range(k+1,len(self.data)):
                    if self.data[l].startswith('>') == False:
                        join_+=1
                    else:
                        break
                acc_seq.append(''.join(self.data[k+1:k+1+join_]))
        return acc_seq

class Pseaac:
    def __init__(self,filename):
        self.filename=filename
    def collect(self,lamb,w,pro,nf):
        val=[]
        val.append(['#']+[ea for ea in keys.values()]+['\u03BB'+str(eac+1) for eac in range(lamb)])
        for e_seq in range(len(seq)):# single sequence taken for test
            q=self.pse(seq[e_seq],lamb,w,pro,nf)
            tem=acc[e_seq].split(' ')[0][1:]
            val.append([acc[e_seq]]+q)
        df=pd.DataFrame(val[1:],columns=val[0])
#         print('PseAAC feature have been extracted!!!')
        return df
        
    def pse(self,data,lamb,w,pro,nf):
        thet=self.theta(data,lamb,pro)
        deno=1+(w*sum(thet.values()))
        p=[]
        if nf==1:
            norm=(len(data))
        else:
            norm=1
        for u in range(1,21+lamb):
            if u>=1 and u<=20:
    #             print(u,'natural')
                num=data.count(keys[u])/norm # frequency
                p.append(num/deno)
            elif u>=21 and u<=20+lamb:
    #             print(u,'pseudo')
                num=w*thet[u-20]
                p.append(num/deno)
        return p
    
    def theta(self,data,lamb,pro):
        the={}
        for u in range(1,lamb+1):
            the[u]=(1/(len(data)-u))*self.rel_cal(data,u,pro)
        return the
    
    def rel_cal(self,data,v,pro):
        tem=[]
        for u in range(len(data)-v):
            te=[]
            for u1 in pro:
                x=((values[u1][data[u]])-(values[u1][data[u+v]]))**2
                te.append(x)
    #             print(u1,u,u+v,data[u],data[u+v],x)
            tem.append(sum(te)/len(pro))
        return sum(tem)

keys,values=Standard_values('D:/After_4_4_22/Project_1_Part_1/CAZy_23_6_22/7_98_hydrophobicity.csv').get_prop([1,2,3,4,5,6,20,60])
# pseaac_data['Length']=stats.zscore(min_len)

In [4]:
'''Clustering Algorithm'''
import os
import time
import matplotlib.pyplot as plt
from sklearn import cluster, mixture, manifold, decomposition, preprocessing,metrics
import random
from collections import Counter,defaultdict
import copy

class clustering:
    rs=77
#     rn=random.randint(1,99)
    def __init__(self,folder,data,n,cat):
        self.x=data.iloc[:,1:].values
        self.y=data.iloc[:,0]
        self.folder=folder
        self.cat=cat
        self.anno_label={0:'acc',1:'prtn',2:'ec',3:'org',4:'species',5:'ghf'}
        temp=[i.split('$')[cat[0]] for i in self.y]
        lab=list(set(temp))
        self.true_lab=[lab.index(j) for j in temp]
        try:
            n.isalpha()
            self.n=len(lab)
        except AttributeError:
            self.n=n
        self.temp1=','.join([f'{k}:{v}' for k,v in dict(Counter(temp)).items()])
        
    def kmeans(self):
        start = time.time()
        self.names = 'km'
        kmeans = cluster.KMeans(n_clusters=self.n,random_state=clustering.rs) # Number of clusters
        self.labels = kmeans.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def affinity(self):
        start = time.time()
        self.names = 'apc'
        apc = cluster.AffinityPropagation(random_state=clustering.rs)
        self.labels = apc.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def meanshift(self):
        start = time.time()
        self.names = 'ms'
        ms = cluster.MeanShift()
        self.labels = ms.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def spectral(self):
        start = time.time()
        self.names = 'spec'
        spectral = cluster.SpectralClustering(n_clusters=self.n,affinity='nearest_neighbors',assign_labels="discretize",random_state=clustering.rs) # Number of clusters
        self.labels = spectral.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def agglomerative(self):
        start = time.time()
        self.names = 'agglo'
        agglo = cluster.AgglomerativeClustering(n_clusters=self.n) # Number of clusters
        self.labels = agglo.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def dbscan(self):
        start = time.time()
        self.names = 'dbs'
        new_x=preprocessing.StandardScaler().fit_transform(self.x)
        dbs = cluster.DBSCAN()
        self.labels = dbs.fit_predict(new_x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def optics(self):
        start = time.time()
        self.names = 'opt'
        opt = cluster.OPTICS()
        self.labels = opt.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def gaussian(self):
        start = time.time()
        self.names = 'gm'
        gm = mixture.GaussianMixture(n_components=self.n,random_state=clustering.rs) # Number of Clusters
        self.labels = gm.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()
        
    def birch(self):
        start = time.time()
        self.names = 'bir'
        brc = cluster.Birch() # Number of clusters
        self.labels = brc.fit_predict(self.x)
        end = time.time()
        self.t = round((end-start),3)
        return self.label_save()

    def label_save(self):
        dfout = pd.DataFrame({'Accession': self.y,  'predicted': self.labels, 'expected':self.true_lab})
        try:
            os.mkdir(self.folder)
        except FileExistsError:
            pass
        dfout.to_csv(f'{self.folder}\ML_{self.names}_{len(set(self.labels))}.txt',sep='\t', index=False)
        self.file()
        return self.analysis()
    
    def analysis(self):
        value=metrics.fowlkes_mallows_score(self.true_lab,self.labels)
        tot_val=[self.names,self.anno_label[self.cat[0]],str(lambda_value),str(round(value,3)),str(self.n),str(len(set(self.labels))),self.temp1,str(len(self.true_lab))]
        return tot_val
    
    def file(self):
        try:
            os.mkdir(f'{self.folder}/table')
        except FileExistsError:
            pass
        dd=defaultdict(list)
        for i in range(len(self.labels)):
            dd[self.labels[i]].append(self.y[i])
        self.excel(dict(sorted(dd.items())))
        
    def excel(self,anno):
        all_anno={}
        for i in self.cat:
            temp={}
            for j,k in anno.items():
                te=[]
                for l in k:
                    te.append(l.split('$')[i])
                temp[j]=dict(Counter(te))
            df=pd.DataFrame(temp).fillna(0).astype(int)
            df.loc['Total']=df.sum(axis=0)
            df.loc[:,'Total']=df.sum(axis=1)
            df.to_excel(f'{self.folder}/table/{self.names}_{len(set(self.labels))}_{self.anno_label[i]}.xlsx')

In [8]:
class ec_data:
    def __init__(self,file):
        self.data=[]
        with open(file,'r') as inpt:
            for i in inpt:
                self.data.append(i.rstrip())

    def cazy(self,dom,typ):
        temp=[]
        for i in self.data:
            temp.append(i.split('$')[1].split(' '))
        return self.domain(temp,dom,typ)
    
    def domain(self,ec_n,n,include):
        single,multi=[],[]
        for i in ec_n:
            if len(i)<=n:
                single.extend(i)
            else:
                multi.append(i)
        print('Number of single domain:',len(single))
        print(f'Number of multi domain (>{n}):',len(multi))
        if include=='m':
            for j in multi:
                for k in j:
                    single.append(k)
            print('After including multi domains:',len(single))
        return single
    
class analysis:
    def __init__(self,data):
        self.data=data
        
    def non_kegg_count(self):
        temp=Counter(self.data)
        return self.specific_ec(dict(temp))
    
    def kegg_count(self,label):
        temp=defaultdict(int)
        for j,k in zip(self.data,label):
            temp[j]+=int(k)
        return self.specific_ec(dict(temp))
    
    def specific_ec(self,dat):
        caazy=[]
        for u,v in dat.items():
            if '3.2.1.' in u:
                try:
                    caazy.append([u,int(u.split('.')[-1])])
                except ValueError:
                    print(u)
        cc_ec=[x[0] for x in sorted(caazy, key=lambda x:x[1])]
        return cc_ec

In [5]:
class add_filter:
    def __init__(self,ac,se):
        self.ac=ac
        self.se=se
    def present_absent(self,typ,data):
        print('Filter has been applied....')
        new_acc,new_seq=[],[]
        not_acc,not_seq=[],[]
        for i in range(len(self.ac)):
            if self.ac[i].split('$')[-1] in data:
                new_acc.append(self.ac[i])
                new_seq.append(self.se[i])
            else:
                not_acc.append(self.ac[i])
                not_seq.append(self.se[i])
        if typ=='present':
            return new_acc,new_seq
        elif typ=='absent':
            return not_acc,not_seq
        else:
            print('Check the spelling!!!')

In [13]:
# replace is applied because "" are coming after opening the files in excel
class selected_GH:
    def __init__(self,file):
        self.data={}
        with open(file,'r') as inpt:
            for i in inpt:
                self.data[i.split('\t')[0]]={j.split(':')[0]:int(j.split(':')[1]) if j.startswith('GH') else 0 for j in i.split('\t')[7].replace('"','').split(',')}
        del self.data['EC_number']
    
    def output(self):
        return self.data

selected_data=selected_GH('EC_18_14_21_17.txt').output()  

In [14]:
selected_data

{'3.2.1.8': {'GH10': 392, 'GH11': 396},
 '3.2.1.14': {'GH18': 496, 'GH19': 196},
 '3.2.1.21': {'GH1': 361, 'GH3': 208},
 '3.2.1.17': {'GH22': 142, 'GH23': 53, 'GH24': 38, 'GH25': 90}}

In [89]:
import random
class balance_data:
    def __init__(self,ac,se):
        self.ac=ac
        self.se=se
        
    def sizing(self,size_data):
        new_ac,new_se=[],[]
        min_size=min(list(size_data.values()))
        for i in size_data:
            temp_ac,temp_se=[],[]
            for j in range(len(self.ac)):
                if i==self.ac[j].split('$')[-1]:
                    temp_ac.append(self.ac[j])
                    temp_se.append(self.se[j])
            new_cord=random.sample([m for m in range(len(temp_ac))],k=min_size)
            for n in new_cord:
                new_ac.append(temp_ac[n])
                new_se.append(temp_se[n])
        print('The data has been balanced to equal size.....')
        return new_ac,new_se 

In [23]:
# '1' stands for single domain, and 'm' stands for to include mutli domain in single domain sequences

######## Chnage the folder name before running the programm ############
main_file='EC_18_14_21_17'
os.mkdir(main_file)
    
# a_cazy=ec_data('D:/After_4_4_22/Project_1_Part_1/CAZy_23_6_22/cazy_char_10_6_22.txt').cazy(1,'s')
# c_cazy=analysis(a_cazy).non_kegg_count()

# for y,z in selected_data.items():
#     print(y,list(z.keys()))

t1=time.perf_counter()
total_data=[]
total_rm_cazy,total_rm_genbank={},{}
value_error,no_entry=[],[]
multi_gh={}

for each,peach in selected_data.items():
    ec_number=each
    gh_family='all'
    cazy_acc_seq,rm_cazy,rm_genbank=In_data.EC_GH(ec_number,gh_family) # write all to fetch all the EC number or all the GH family
    total_rm_cazy[each],total_rm_genbank[each]=rm_cazy,rm_genbank
    acc,seq,min_len,x_aa,dupli,multi_gh[each]=Sequence(cazy_acc_seq).output()
    print(f'Sequence has been collected for {each}....')
    
######## Filtered applied #################################

    acc,seq=add_filter(acc,seq).present_absent('present',list(peach.keys()))
    
######## Taking equal size GH families #####################

#     acc,seq=balance_data(acc,seq).sizing(peach)
    
    min_lamb=30
    try:
        if min(min_len)<min_lamb:
            lambda_value=min(min_len)-1
        else:
            lambda_value=min_lamb
    except ValueError:
        print(each,': doesnt have sequences')
        no_entry.append(each)
        continue
    pseaac_data=Pseaac(f'PAAC_{ec_number}_{gh_family}_L{lambda_value}.txt').collect(lambda_value,0.05,[60,2,3],1)
    print(f'Feature has been extracted for {each}....')
    ec_=ec_number.replace('.','_')
    # In the below statement 'auto' means it takes automatic clusters based on number labels given, you can also choose any number.
    clust=clustering(f'{main_file}/{gh_family}_{ec_}',pseaac_data,'auto',[5]) # 0:'acc',1:'prtn',2:'ec',3:'org',4:'species',5:'ghf'
    try:
        km=clust.kmeans()
        apc=clust.affinity()
        ms=clust.meanshift()
        spec=clust.spectral()
        agglo=clust.agglomerative()
        dbs=clust.dbscan()
        opt=clust.optics()
        gm=clust.gaussian()
        bir=clust.birch()
    except ValueError:
        print(f'{each} has {len(acc)} samples which is less than 5 min_samples ')
        value_error.append([each,len(acc)])
        continue
    print(f'Clustering has been done for {each}....')
    all_clust={'km':km,'apc':apc,'ms':ms,'spec':spec,'agglo':agglo,'dbs':dbs,'opt':opt,'gm':gm,'bir':bir}
    for aa in all_clust.values():
        bb=[each]+aa+[str(len(rm_cazy)),str(len(rm_genbank))]+[str(x_aa)]+[str(dupli),str(len(multi_gh[each]))]
        total_data.append('$'.join(bb))
t2=time.perf_counter()
print('Mission completed in',round(t2-t1,3),'seconds')

No hits for BAA31551.1
No hits for NP_149279.1
No hits for NP_149217.1
No hits for AAC98123.1
No hits for ABI49937.1
No hits for AAC98140.1
No hits for CAA82319.1
No hits for NP_242986.1
No hits for ZP_08160171.1
No hits for ZP_08159615.1
No hits for ACR61562.1
No hits for BAA19777.1
No hits for ADQ57411.1
No hits for YP_003851606.1
No hits for AAZ56824.1
No hits for AAZ56956.1
No hits for CAD48748.1
No hits for NP_227886.1
No hits for NP_227877.1
No hits for AAD32593.1
No hits for NP_644553.1
No hits for AAP31839.1
No hits for EAA78230.1
No hits for XP_002470423.1
No hits for XP_365543.1
No hits for BAA89465.1
No hits for NP_241765.1
No hits for CAD65888.1
No hits for ZP_08160424.1
No hits for ZP_08158180.1
No hits for ZP_08157788.1
No hits for NP_624448.1
No hits for NP_626540.1
No hits for BAA19778.1
No hits for AAZ55251.1
No hits for ABA39289.1
No hits for ABM55503.1
No hits for EAA73188.1
No hits for XP_383800.1
No hits for XP_368051.1
No hits for AAD37441.1
No hits for A44594
No 

c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_affinity_propagation.py:246: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_birch.py:647: ConvergenceWarning: Number of subclusters found (1) by Birch is less than (3). Decrease the threshold.
  warnings.warn(


Clustering has been done for 3.2.1.8....
No hits for NP_279794.1
No hits for NP_578962.1
No hits for NP_578963.1
No hits for NP_578962.1
No hits for NP_833450.1
No hits for AAU21943.1
No hits for NP_902605.1
No hits for AGD94964.1
No hits for NP_814153.1
No hits for NP_241782.1
No hits for NP_268107.1
No hits for CBW16119.1
No hits for ABV39247.1
No hits for ABV40327.1
No hits for ABV41826.1
No hits for ABV42574.1
No hits for NP_630126.1
No hits for NP_629155.1
No hits for NP_629515.1
No hits for NP_625711.1
No hits for AAZ54618.1
No hits for NP_232428.1
No hits for AAA83586.1
No hits for EAL00460.1
No hits for ABC59330.1
No hits for AGX26690.1
No hits for AAH36339.1
No hits for AAH47336.1
No hits for CAH70802.1
No hits for CAH70803.1
No hits for CAH70804.1
No hits for CAI19263.1
No hits for CAI19265.1
No hits for CAI19266.1
No hits for NP_068569.1
No hits for BAB25878.1
No hits for BAB90566.1
No hits for BAB91759.1
No hits for ABB97081.1
No hits for AAV58834.1
No hits for BAA31200.1
N

c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_affinity_propagation.py:246: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_birch.py:647: ConvergenceWarning: Number of subclusters found (1) by Birch is less than (3). Decrease the threshold.
  warnings.warn(


Clustering has been done for 3.2.1.14....
No hits for NP_578171.1
No hits for NP_142340.1
No hits for ABP70047.1
No hits for YP_066184.1
No hits for NP_242789.1
No hits for ABR73190.1
No hits for NP_386997.1
No hits for NP_625353.1
No hits for NP_631601.1
No hits for AAZ54975.1
No hits for ADI56259.1
No hits for NP_936184.1
No hits for AEE33889.1
No hits for NP_176375.1
No hits for NP_187303.1
No hits for AAG12895.1
No hits for NP_198505.1
No hits for AAA83309.1
No hits for AAC68766.1
No hits for NP_497558.1
No hits for ABI34907.1
No hits for ABI34907.2
No hits for ACD65509.1
No hits for EAA26947.1
No hits for XP_322216.1
No hits for BAB86071.1
No hits for BAC06894.1
No hits for BAF07003.1
No hits for CAE05491.1
No hits for CAE03398.1
No hits for CAE01908.1
No hits for CAE01909.1
No hits for CAE01911.1
No hits for AAA84906.2
No hits for AAT85322.1
No hits for AAL89551.1
No hits for NP_812226.1
No hits for YP_002352162.1
No hits for ZP_01543735.1
No hits for NP_227841.1
No hits for XP_3

c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_affinity_propagation.py:246: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_birch.py:647: ConvergenceWarning: Number of subclusters found (1) by Birch is less than (3). Decrease the threshold.
  warnings.

Clustering has been done for 3.2.1.21....
No hits for AAC00558.1
No hits for ABK34500.1
No hits for XP_046047.1
No hits for NP_815238.1
No hits for NP_420976.1
No hits for NP_845781.1
No hits for NP_845154.1
No hits for NP_347193.1
No hits for NP_562230.1
No hits for NP_815667.1
No hits for NP_814147.1
No hits for BAB45607.1
Sequence has been collected for 3.2.1.17....
Filter has been applied....
Feature has been extracted for 3.2.1.17....
Clustering has been done for 3.2.1.17....
Mission completed in 122.91 seconds


c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\users\rakes\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_birch.py:647: ConvergenceWarning: Number of subclusters found (1) by Birch is less than (3). Decrease the threshold.
  warnings.warn(


In [24]:
from tqdm import tqdm
titl=['EC_number','Method','label_type','lambda','FMI','ex_groups','pred_groups','distribution','total','CAZy_partial','Fasta_partial','X_aa','Duplicates','Multi_GH_fam']
outpt=open('Summary_ec_number_18_14_21_17.txt','w')
outpt.write('$'.join(titl)+'\n')
for line in tqdm(total_data):
    outpt.write(line+'\n')
outpt.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<?, ?it/s]
